In [ ]:
#default_exp query
%load_ext autoreload
%autoreload 2

# Queries

> Este módulo executa as queries sql / MongoDB necessárias para baixar os dados do STEL, RADCOM e MOSAICO

In [ ]:
#export
import requests
from decimal import *
from typing import *
from gazpacho import Soup
from rich.progress import track
from pathlib import Path
from unidecode import unidecode
import pandas as pd
import pandas_read_xml as pdx
import pyodbc
import re
import xml.etree.ElementTree as et
from zipfile import ZipFile
import collections
from fastcore.utils import listify
from fastcore.foundation import L
from anateldb.constants import console
getcontext().prec = 5

In [ ]:
#export
TIMEOUT = 5
RELATORIO = "http://sistemas.anatel.gov.br/se/eApp/reports/b/srd/resumo_sistema.php?id={id}&state={state}"
ESTACOES = "http://sistemas.anatel.gov.br/se/public/file/b/srd/estacao_rd.zip"
ESTACAO = "http://sistemas.anatel.gov.br/se/public/view/b/srd.php?wfid=estacoes&id={}"
PLANO_BASICO = "http://sistemas.anatel.gov.br/se/public/file/b/srd/Canais.zip"
HISTORICO = (
    "http://sistemas.anatel.gov.br/se/public/file/b/srd/documento_historicos.zip"
)
REJECT_ESTACOES = [
    "atenuacao",
    "historico_documentos",
    "estacao_auxiliar",
    "rds",
    "aprovacao_locais",
    "item",
]
COL_ESTACOES = (
    "siglaservico",
    "num_servico",
    "state",
    "entidade",
    "fistel",
    "municipio",
    "uf",
    "id",
    "numero_estacao",
    "latitude",
    "longitude",
    "cnpj",
    "habilitacao_datavalfreq",
)
NEW_ESTACOES = [
    "Serviço",
    "Num_Serviço",
    "Status",
    "Entidade",
    "Fistel",
    "Município",
    "UF",
    "Id",
    "Número_da_Estação",
    "Latitude_Transmissor",
    "Longitude_Transmissor",
    "CNPJ",
    "Validade_RF",
    "Num_Ato",
    "Data_Ato",
]
COL_PB = (
    "id",
    "municipio",
    "frequencia",
    "classe",
    "servico",
    "entidade",
    "latitude",
    "longitude",
    "uf",
    "status",
    "cnpj",
    "fistel",
    'canal'
)
NEW_PB = (
    "Id",
    "Município",
    "Frequência",
    "Classe",
    "Serviço",
    "Entidade",
    "Latitude_Estação",
    "Longitude_Estação",
    "UF",
    "Status",
    "CNPJ",
    "Fistel",
    "Canal"
)
TELECOM = (
    "Frequência",
    "Serviço",
    "Entidade",
    "Fistel",
    "Número da Estação",
    "Município",
    "UF",
    "Latitude",
    "Longitude",
    "Validade_RF"
)
RADIODIFUSAO = (
    "Frequência",
    "Num_Serviço",
    "Status",
    "Classe",
    "Entidade",
    "Fistel",
    "Número_da_Estação",
    "Município",
    "UF",
    "Latitude",
    "Longitude",
    "Validade_RF",
    "Num_Ato",
    "Data_Ato",
)

APP_ANALISE = (
    "Frequency",
    "Latitude",
    "Longitude",
    "Description",
    "Service",
    "Station",
    "ActNumber",
    "ActDate",
    "ValRF",
)

ENTIDADES = {}

In [ ]:
#export
RADCOM = """
       select f.MedFrequenciaInicial as 'Frequência',
       Sitarweb.dbo.FN_SRD_RetornaIndFase(PB.NumServico, Pr.idtPedidoRadcom) as 'Fase',
       Sitarweb.dbo.FN_SRD_RetornaSiglaSituacao(h.IdtHabilitacao, Es.IdtEstacao) as 'Situação',
       uf.SiglaUnidadeFrequencia as 'Unidade',
       e.NomeEntidade as 'Entidade',
       h.NumFistel as 'Fistel',
       es.NumEstacao as 'Número da Estação',
       m.NomeMunicipio as 'Município',
       pb.SiglaUF as 'UF',
       es.MedLatitudeDecimal as 'Latitude',
       es.MedLongitudeDecimal as 'Longitude',
       e.NumCnpjCpf as 'CNPJ'
from ENTIDADE e
inner join HABILITACAO h on h.IdtEntidade = e.IdtEntidade
inner join SRD_PEDIDORADCOM pr on pr.IdtHabilitacao = h.IdtHabilitacao
inner join SRD_PLANOBASICO pb on pb.IdtPlanoBasico = pr.IdtPlanoBasico
inner join estacao es on es.IdtHabilitacao = h.IdtHabilitacao
inner join FREQUENCIA f on f.IdtEstacao = es.IdtEstacao
inner join UnidadeFrequencia uf on uf.IdtUnidadeFrequencia = f.IdtUnidadeFrequencia
inner join Municipio m on m.CodMunicipio = pb.CodMunicipio
where h.NumServico = '231'
"""

In [ ]:
#export
STEL = """IF OBJECT_ID('tempDB..##faixas','U') is not null
drop table ##faixas
create table ##faixas (id int not null, faixa varchar(20), inic float, fim float,);
insert into ##faixas values(0,'De 20 MHz - 6 GHz','20000', '6000000');

select distinct f.MedTransmissaoInicial as 'Frequência',
uf.SiglaUnidadeFrequencia as 'Unidade',
e.NumServico as 'Serviço',
ent.NomeEntidade as 'Entidade',
h.NumFistel as 'Fistel',
e.NumEstacao as 'Número da Estação',
mu.NomeMunicipio as 'Município',
e.SiglaUf as 'UF',
e.MedLatitudeDecimal as 'Latitude',
e.MedLongitudeDecimal as 'Longitude',
ent.NumCnpjCpf as 'CNPJ',
c.DataValidadeRadiofrequencia as 'Validade_RF'
from contrato c
inner join estacao e on e.IdtContrato = c.Idtcontrato
inner join frequencia f on f.IdtEstacao = e.IdtEstacao
inner join HABILITACAO h on h.IdtHabilitacao = c.IdtHabilitacao
inner join entidade ent on ent.IdtEntidade = h.IdtEntidade
inner join endereco en on en.IdtEstacao = e.IdtEstacao
inner join Municipio mu on mu.CodMunicipio = en.CodMunicipio
inner join Servico s on s.NumServico = h.NumServico and s.IdtServicoAreaAtendimento = 4
left join UnidadeFrequencia uf on uf.IdtUnidadeFrequencia = f.IdtUnidadeTransmissao
left outer join ##faixas fx on (
(fx.inic <= f.MedRecepcaoInicialKHz and fx.fim >= f.MedRecepcaoInicialKHz)
or (fx.inic <= f.MedTransmissaoInicialKHz and fx.fim >= f.medtransmissaoinicialkhz)
or (fx.inic <= f.MedFrequenciaInicialKHz and fx.fim >= f.MedFrequenciaInicialKHz)
or (fx.inic <= f.MedFrequenciaFinalKHz and fx.fim >= f.MedFrequenciaFinalKHz)
)
where e.DataExclusao is null and
fx.faixa is not null and
f.MedTransmissaoInicial is not null
and h.NumServico <> '010'
"""

## Otimização dos Tipos de dados
A serem criados dataframes, normalmente a tipo de data é aquele com maior resolução possível, nem sempre isso é necessário, os arquivos de espectro mesmo possuem somente uma casa decimal, portanto um `float16` já é suficiente para armazená-los. As funções a seguir fazem essa otimização

Code below borrowed from https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2

In [ ]:
#export
def optimize_floats(df: pd.DataFrame, exclude = None) -> pd.DataFrame:
    floats = df.select_dtypes(include=["float64"]).columns.tolist()
    floats = [c for c in floats if c not in listify(exclude)]
    df[floats] = df[floats].apply(pd.to_numeric, downcast="float")
    return df


def optimize_ints(df: pd.DataFrame, exclude=None) -> pd.DataFrame:
    ints = df.select_dtypes(include=["int64"]).columns.tolist()
    ints = [c for c in ints if c not in listify(exclude)]
    df[ints] = df[ints].apply(pd.to_numeric, downcast="integer")
    return df


def optimize_objects(df: pd.DataFrame, datetime_features: List[str], exclude) -> pd.DataFrame:
    for col in df.select_dtypes(include=["object"]):
        if col not in datetime_features:
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                if col in listify(exclude): continue
                df[col] = df[col].astype("category")
        else:
            df[col] = pd.to_datetime(df[col]).dt.date
    return df


def df_optimize(df: pd.DataFrame, datetime_features: List[str] = [], exclude = None):
    return optimize_floats(optimize_ints(optimize_objects(df, datetime_features, exclude), exclude), exclude)

In [ ]:
#export
def connect_db():
    """Conecta ao Banco ANATELBDRO01 e retorna o 'cursor' (iterador) do Banco pronto para fazer iterações"""
    conn = pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=ANATELBDRO01;"
        "Database=SITARWEB;"
        "Trusted_Connection=yes;"
        "MultipleActiveResultSets=True;",
        timeout=TIMEOUT,
    )
    return conn

In [ ]:
#slow
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (RADCOM, STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

In [ ]:
#export
def row2dict(row):
    """Receives a json row and return the dictionary from it"""
    return {k: v for k, v in row.items()}


def dict2cols(df, reject=()):
    """Recebe um dataframe com dicionários nas células e extrai os dicionários como colunas
    Opcionalmente ignora e exclue as colunas em reject
    """
    for column in df.columns:
        if column in reject:
            df.drop(column, axis=1, inplace=True)
            continue
        if type(df[column].iloc[0]) == collections.OrderedDict:
            try:
                new_df = pd.DataFrame(df[column].apply(row2dict).tolist())
                df = pd.concat([df, new_df], axis=1)
                df.drop(column, axis=1, inplace=True)
            except AttributeError:
                continue
    return df


def parse_plano_basico(row, cols=COL_PB):
    """Receives a json row and filter the column in `cols`"""
    return {k: row[k] for k in cols}


def scrape_dataframe(id_list):
    df = pd.DataFrame()
    for id_ in track(id_list, description="Baixando informações complementares da Web"):
        html = requests.get(ESTACAO.format(id_))
        df = df.append(pd.read_html(Soup(html.text).find("table").html)[0])
    
    df.rename(columns={'NumFistel': 'Fistel',
                       'Num Serviço': 'Num_Serviço'}, inplace=True)
    return df[["Status", "Entidade", "Fistel", "Frequência", "Classe", 'Num_Serviço', 'Município', 'UF']]

In [ ]:
#export
def read_stel(pasta, update=False):
    """Lê o banco de dados salvo localmente do STEL. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01."""
    if update:
        update_stel(pasta)
    if (file := Path(f"{pasta}/stel.fth")).exists():
        stel = pd.read_feather(file)
    elif (file := Path(f"{pasta}/stel.xlsx")).exists():
        stel = pd.read_excel(file, engine="openpyxl")
    else:
        update_stel(pasta)
        try:
            stel = pd.read_feather(Path(f"{pasta}/stel.fth"))
        except FileNotFoundError as e:
            raise ConnectionError(
                "Base de Dados do Stel inexistente e não foi possível atualizá-la"
            ) from e
    return stel


def read_radcom(pasta, update=False):
    """Lê o banco de dados salvo localmente de RADCOM. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01."""
    if update:
        update_radcom(pasta)
    if (file := Path(f"{pasta}/radcom.fth")).exists():
        radcom = pd.read_feather(file)        
    elif (file := Path(f"{pasta}/radcom.xlsx")).exists():
        radcom = pd.read_excel(file, engine="openpyxl")
    elif (file := Path(f"{pasta}/radcom.csv")).exists():
        radcom = pd.read_csv(file)
    else:
        update_radcom(pasta)
        try:
            radcom = pd.read_feather(Path(f"{pasta}/radcom.fth"))
        except FileNotFoundError as e:
            raise ConnectionError(
                "Base de Dados do Stel inexistente e não foi possível atualizá-la"
            ) from e
    return radcom


def read_estações(path):
    def extrair_ato(row):
        if not isinstance(row, str):
            row = listify(row)[::-1]
            for d in row:
                if not isinstance(d, dict):
                    continue
                if (d.get("@TipoDocumento") == "Ato") and (
                    d.get("@Razao") == "Autoriza o Uso de Radiofrequência"
                ):
                    return d["@NumDocumento"], d["@DataDOU"][:10]
            else:
                return "", ""
        return "", ""

    es = pdx.read_xml(path, ["estacao_rd"])
    dfs = []
    for i in range(es.shape[0]):
        df = pd.DataFrame(es["row"][i]).replace("", pd.NA)
        df = dict2cols(df)
        df.columns = [unidecode(c).lower().replace("@", "") for c in df.columns]
        dfs.append(df)
    df = pd.concat(dfs)
    df = df[df.state.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(drop=True)
    docs = L(df.historico_documentos.apply(extrair_ato).tolist())
    df = df.loc[:, COL_ESTACOES]
    df["Num_Ato"] = docs.itemgot(0).map(str)
    df["Data_Ato"] = docs.itemgot(1).map(str)
    df.columns = NEW_ESTACOES
    df['Validade_RF'] = df.Validade_RF.astype('str').str.slice(0,10)
    df["Data_Ato"] = df.Data_Ato.str.slice(0,10)
    df['Entidade'] = df.Entidade.fillna('')
    ENTIDADES.update({r.Fistel : r.Entidade for r in df.itertuples() if r.Entidade != ''})
    return df


def read_plano_basico(path):
    pb = pdx.read_xml(path, ["plano_basico"])
    # df = pd.DataFrame(df["row"].apply(row2dict).tolist()).replace("", pd.NA)
    dfs = []
    for i in range(pb.shape[0]):
        df = pd.DataFrame(pb["row"][i]).replace("", pd.NA)
        df = dict2cols(df)
        df.columns = [unidecode(c).lower().replace("@", "") for c in df.columns]
        dfs.append(df)
    df = pd.concat(dfs)
    df = df.loc[df.pais == "BRA", COL_PB].reset_index(drop=True)
    df.columns = NEW_PB
    df.sort_values(["Id", "Canal"], inplace=True)
    df['Entidade'] = df.Entidade.fillna('')
    ENTIDADES.update({r.Fistel : r.Entidade for r in df.itertuples() if r.Entidade != ''})
    df = df.groupby("Id", as_index=False).first()  # remove duplicated with NaNs
    df.dropna(subset=['Status'], inplace=True)
    df = df[df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(drop=True)
    return df

def read_historico(path):
    regex = r"\s([a-zA-Z]+)=\'{1}([\w\-\ :\.]*)\'{1}"
    with ZipFile(path) as xmlzip:
        with xmlzip.open("documento_historicos.xml", "r") as xml:
            xml_list = xml.read().decode().split("\n")[2:-1]
    dict_list = []
    for item in xml_list:
        matches = re.finditer(regex, item, re.MULTILINE)
        dict_list.append(dict(match.groups() for match in matches))
    df = pd.DataFrame(dict_list)
    df = df[
        (df.tipoDocumento == "Ato") & (df.razao == "Autoriza o Uso de Radiofrequência")
    ].reset_index()
    df = df.loc[:, ["id", "numeroDocumento", "orgao", "dataDocumento"]]
    df.columns = ["Id", "Num_Ato", "Órgao", "Data_Ato"]
    df["Data_Ato"] = pd.to_datetime(df.Data_Ato)
    return df.sort_values("Data_Ato").groupby("Id").last().reset_index()


def read_mosaico(pasta, update=False):
    if update:
        update_mosaico(pasta)
    if (file := Path(f"{pasta}/mosaico.fth")).exists():
        return pd.read_feather(file)
    elif (file := Path(f"{pasta}/mosaico.xlsx")).exists():
        return pd.read_excel(f"{pasta}/mosaico.xlsx")
    return read_mosaico(pasta, update=True)

def read_base(pasta, up_stel=False, up_radcom=False, up_mosaico=False):
    if any([up_stel, up_radcom, up_mosaico]):
        update_base(pasta, up_stel, up_radcom, up_mosaico)
    if (file := Path(f"{pasta}/base.fth")).exists():
        return pd.read_feather(file)
    elif (file := Path(f"{pasta}/base.xlsx")).exists():
        return pd.read_excel(file, engine='openpyxl')
    else:
        update_base(pasta, up_stel, up_radcom, up_mosaico)
        read_base(pasta, False, False, False)

In [ ]:
#export
def clean_merge(pasta, df):
    df = df.copy()
    COLS = [c for c in df.columns if "_x" in c]
    for col in COLS:
        col_x = col
        col_y = col.split("_")[0] + "_y"
        if df[col_x].count() > df[col_y].count():
            a, b = col_x, col_y
        else:
            a, b = col_y, col_x

        df.loc[df[a].isna(), a] = df.loc[df[a].isna(), b]
        df.drop(b, axis=1, inplace=True)
        df.rename({a: a[:-2]}, axis=1, inplace=True)

    df.loc[df.Latitude_Transmissor == "", "Latitude_Transmissor"] = df.loc[
        df.Latitude_Transmissor == "", "Latitude_Estação"
    ]
    df.loc[df.Longitude_Transmissor == "", "Longitude_Transmissor"] = df.loc[
        df.Longitude_Transmissor == "", "Longitude_Estação"
    ]
    df.loc[df.Latitude_Transmissor.isna(), "Latitude_Transmissor"] = df.loc[
        df.Latitude_Transmissor.isna(), "Latitude_Estação"
    ]
    df.loc[df.Longitude_Transmissor.isna(), "Longitude_Transmissor"] = df.loc[
        df.Longitude_Transmissor.isna(), "Longitude_Estação"
    ]
    df.drop(["Latitude_Estação", "Longitude_Estação"], axis=1, inplace=True)
    df.rename(
        columns={
            "Latitude_Transmissor": "Latitude",
            "Longitude_Transmissor": "Longitude",
        },
        inplace=True,
    )
    m = pd.read_excel(f"{pasta}/municípios.xlsx", engine='openpyxl')
    m.loc[
        m.Município == "Sant'Ana do Livramento", "Município"
    ] = "Santana do Livramento"
    m["Município"] = (
        m.Município.apply(unidecode).str.lower().str.replace("'", " ")
    )  # (lambda x: "".join(e for e in x if e.isalnum()))
    m["UF"] = m.UF.str.lower()
    df["Coordenadas_do_Município"] = False
    df["Latitude"] = df.Latitude.str.replace(",", ".")
    df["Longitude"] = df.Longitude.str.replace(",", ".")
    df["Frequência"] = df.Frequência.str.replace(",", ".")
    df.loc[df["Município"] == "Poxoréo", "Município"] = "Poxoréu"
    df.loc[df["Município"] == "Couto de Magalhães", "Município"] = "Couto Magalhães"
    for row in df[(df.Latitude == "") | (df.Latitude.isna())].itertuples():
        try:
            left = unidecode(row.Município).lower()
            m_coord = (
                m.loc[
                    (m.Município == left) & (m.UF == row.UF.lower()),
                    ["Latitude", "Longitude"],
                ]
                .values.flatten()
                .tolist()
            )
            df.loc[row.Index, "Latitude"] = m_coord[0]
            df.loc[row.Index, "Longitude"] = m_coord[1]
            df.loc[row.Index, "Coordenadas_do_Município"] = True
        except ValueError:
            print(left, row.UF, m_coord)
            continue

    freq_nans = df[df.Frequência.isna()].Id.tolist()
    complement_df = scrape_dataframe(freq_nans)
    df.loc[
        df.Frequência.isna(), ["Status", "Entidade", "Fistel", "Frequência", "Classe", 
                               'Num_Serviço', 'Município', 'UF']
        ] = complement_df.values
        
    for r in df[(df.Entidade.isna()) | (df.Entidade == '')].itertuples():
        df.loc[r.Index, 'Entidade'] = ENTIDADES.get(r.Fistel, '')

    df.loc[df["Número_da_Estação"] == "", "Número_da_Estação"] = -1
    df["Latitude"] = df["Latitude"].astype("float")
    df["Longitude"] = df["Longitude"].astype("float")
    df["Frequência"] = df.Frequência.astype("float")
    df.loc[df.Serviço == 'OM', 'Frequência'] = df.loc[df.Serviço == 'OM', 'Frequência'].apply(lambda x: Decimal(x) / Decimal(1000))
    df["Frequência"] = df.Frequência.astype("float")
#     return df_optimize(df, exclude=['Latitude', 'Longitude', 'Frequência', 'Validade_RF'])
    return df

In [ ]:
#export
def update_radcom(folder):
    """Update the Radcom File querying the Database"""
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="earth"
    ) as status:
        try:
            conn = connect_db()
            df = pd.read_sql_query(RADCOM, conn)
            df = df_optimize(df, exclude=['Frequência', 'Latitude', 'Longitude'])
            df.to_feather(f"{folder}/radcom.fth")
            df.to_excel(f"{folder}/radcom.xlsx", engine='openpyxl', index=False)

        except pyodbc.OperationalError:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )


def update_stel(folder):
    """Update the Stel File querying the Database"""
    with console.status(
        "[magenta]Lendo o Banco de Dados do STEL. Processo Lento, aguarde...",
        spinner="moon",
    ) as status:
        try:
            conn = connect_db()
            df = pd.read_sql_query(STEL, conn)
            df['Validade_RF'] = df.Validade_RF.astype('str').str.slice(0,10)
            df = df_optimize(df, exclude=['Frequência', 'Latitude', 'Longitude'])
            df.to_feather(f"{folder}/stel.fth")
            df.to_excel(f"{folder}/stel.xlsx", engine='openpyxl', index=False)
        except pyodbc.OperationalError:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )


def update_mosaico(pasta):
    """Update the Mosaico File by downloading the zipped xml file from the Spectrum E Web page"""
    with console.status(
        "[blue]Baixando as Estações do Mosaico...", spinner="shark"
    ) as status:
        file = requests.get(ESTACOES)
        with open(f"{pasta}/estações.zip", "wb") as estações:
            estações.write(file.content)
    with console.status(
        "[blue]Baixando o Plano Básico das Estações...", spinner="weather"
    ) as status:
        file = requests.get(PLANO_BASICO)
        with open(f"{pasta}/Canais.zip", "wb") as plano_basico:
            plano_basico.write(file.content)
    console.print("[blue]Consolidando as bases de dados...")
    estações = read_estações(f"{pasta}/estações.zip")
    plano_basico = read_plano_basico(f"{pasta}/Canais.zip")
    df = estações.merge(plano_basico, on="Id", how="left")
    df = clean_merge(pasta, df)
    df.reset_index(drop=True).to_feather(f"{pasta}/mosaico.fth")
    with pd.ExcelWriter(f"{pasta}/mosaico.xlsx") as workbook:
        df.reset_index(drop=True).to_excel(workbook, sheet_name='Sheet1', engine="openpyxl", index=False)
    console.print("Kbô :vampire:")
    return df


def update_base(folder, up_stel=False, up_radcom=False, up_mosaico=False):
    stel = read_stel(path, up_stel).loc[:, TELECOM]
    stel.rename(
        columns={"Serviço": "Num_Serviço", "Número da Estação": "Número_da_Estação"},
        inplace=True,
    )
    radcom = read_radcom(path, up_radcom)
    radcom.rename(columns={"Número da Estação": "Número_da_Estação"}, inplace=True)
    mosaico = read_mosaico(path, up_mosaico)
    radcom["Num_Serviço"] = 231
    radcom["Status"] = "RADCOM"
    radcom["Classe"] = radcom.Fase.str.strip() + "-" + radcom.Situação.str.strip()
    radcom["Entidade"] = radcom.Entidade.str.rstrip().str.lstrip()
    radcom["Num_Ato"] = ""
    radcom["Data_Ato"] = ""
    radcom["Validade_RF"] = ""
    radcom = radcom.loc[:, RADIODIFUSAO]
    stel["Num_Ato"] = ""
    stel["Data_Ato"] = ""
    stel['Entidade'] = stel.Entidade.str.rstrip().str.lstrip()
    mosaico = mosaico.loc[:, RADIODIFUSAO]
    rd = mosaico.append(radcom)
    rd = rd.append(stel).sort_values("Frequência").reset_index(drop=True)
#     rd = df_optimize(rd, exclude=['Frequência', 'Latitude', 'Longitude', 'Validade_RF', 'Data_Ato'])
#     rd.to_feather(f"{folder}/base.fth")
    with pd.ExcelWriter(f"{folder}/base.xlsx") as workbook:
        rd.reset_index(drop=True).to_excel(workbook, sheet_name='Sheet1', engine="openpyxl", index=False)
    return rd


In [ ]:
path = r'D:\OneDrive - ANATEL\BaseDados'

In [ ]:
stel = read_stel(path)

In [ ]:
stel = pd.read_excel(f'{path}/stel2.xlsx', header=None, engine='openpyxl')

In [ ]:
stel.columns = ("Frequência",
    "Unidade",
    "Serviço",
    "Entidade",
    "Fistel",
    "Número da Estação",
    "Município",
    "UF",
    "Latitude",
    "Longitude",
    "CNPJ",
    "Validade_RF"
)

In [ ]:
stel.Frequência /= 100000000

In [ ]:
stel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404423 entries, 0 to 404422
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Frequência         404423 non-null  float64
 1   Unidade            404423 non-null  object 
 2   Serviço            404423 non-null  int64  
 3   Entidade           404423 non-null  object 
 4   Fistel             404423 non-null  int64  
 5   Número da Estação  404423 non-null  int64  
 6   Município          404423 non-null  object 
 7   UF                 404423 non-null  object 
 8   Latitude           404423 non-null  float64
 9   Longitude          404423 non-null  float64
 10  CNPJ               404423 non-null  int64  
 11  Validade_RF        391042 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 37.0+ MB


In [ ]:
stel.head()

,Frequência,Unidade,Serviço,Entidade,Fistel,Número da Estação,Município,UF,Latitude,Longitude,CNPJ,Validade_RF
0,29.793,MHz,19,RAFT TECNOLOGIES BRAZIL SISTEMAS DE TECNOLOGIA...,50411558609,1000352878,Santana de Parnaíba,SP,-2.346758e+17,-4.686211e+17,17196674000106,2035-03-11
1,29.799,MHz,35,RAFT TECNOLOGIES BRAZIL SISTEMAS DE TECNOLOGIA...,50411805045,1000487528,São Paulo,SP,-2.366278e+17,-4.669583e+17,17196674000106,2014-12-20
2,33.120,MHz,19,ANTONIO CARLOS URENHA,2021353494,2718600,Serra Azul,SP,-2.126667e+17,-4.758333e+17,16779070882,2040-10-17
3,33.120,MHz,19,ANTONIO CARLOS URENHA,2021353494,2718618,Serra Azul,SP,-2.126667e+17,-4.758333e+17,16779070882,2040-10-17
4,34.480,MHz,78,ASSOCIACAO DOS TAXISTAS E RADIO TAXIS MUNIC SA...,2033423808,690266405,São Paulo,SP,-2.347444e+17,-4.665639e+17,67644641000150,2018-03-11


In [ ]:
stel['Validade_RF'] = stel.Validade_RF.str.slice(0,10)

In [ ]:
stelop = df_optimize(stel.copy(), exclude=['Frequência', 'Latitude', 'Longitude'])

In [ ]:
stelop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404423 entries, 0 to 404422
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Frequência         404423 non-null  float64 
 1   Unidade            404423 non-null  category
 2   Serviço            404423 non-null  int16   
 3   Entidade           404423 non-null  category
 4   Fistel             404423 non-null  int64   
 5   Número da Estação  404423 non-null  int32   
 6   Município          404423 non-null  category
 7   UF                 404423 non-null  category
 8   Latitude           404423 non-null  float64 
 9   Longitude          404423 non-null  float64 
 10  CNPJ               404423 non-null  int64   
 11  Validade_RF        391042 non-null  category
dtypes: category(5), float64(3), int16(1), int32(1), int64(2)
memory usage: 21.8 MB


In [ ]:
stelop.head()

,Frequência,Unidade,Serviço,Entidade,Fistel,Número da Estação,Município,UF,Latitude,Longitude,CNPJ,Validade_RF
0,29.793,MHz,19,RAFT TECNOLOGIES BRAZIL SISTEMAS DE TECNOLOGIA...,50411558609,1000352878,Santana de Parnaíba,SP,-2.346758e+17,-4.686211e+17,17196674000106,2035-03-11
1,29.799,MHz,35,RAFT TECNOLOGIES BRAZIL SISTEMAS DE TECNOLOGIA...,50411805045,1000487528,São Paulo,SP,-2.366278e+17,-4.669583e+17,17196674000106,2014-12-20
2,33.120,MHz,19,ANTONIO CARLOS URENHA,2021353494,2718600,Serra Azul,SP,-2.126667e+17,-4.758333e+17,16779070882,2040-10-17
3,33.120,MHz,19,ANTONIO CARLOS URENHA,2021353494,2718618,Serra Azul,SP,-2.126667e+17,-4.758333e+17,16779070882,2040-10-17
4,34.480,MHz,78,ASSOCIACAO DOS TAXISTAS E RADIO TAXIS MUNIC SA...,2033423808,690266405,São Paulo,SP,-2.347444e+17,-4.665639e+17,67644641000150,2018-03-11


In [ ]:
stelop.sort_values('Frequência')

,Frequência,Unidade,Serviço,Entidade,Fistel,Número da Estação,Município,UF,Latitude,Longitude,CNPJ,Validade_RF
519,29.792999,MHz,19,RAFT TECNOLOGIES BRAZIL SISTEMAS DE TECNOLOGIA...,50411558609,1000352878,Santana de Parnaíba,SP,-23.467583,-46.862110,17196674000106,2035-03-11
675,29.799000,MHz,35,RAFT TECNOLOGIES BRAZIL SISTEMAS DE TECNOLOGIA...,50411805045,1000487528,São Paulo,SP,-23.662777,-46.695835,17196674000106,2014-12-20
1653,33.119999,MHz,19,ANTONIO CARLOS URENHA,2021353494,2718618,Serra Azul,SP,-21.266666,-47.583332,16779070882,2040-10-17
1143,33.119999,MHz,19,ANTONIO CARLOS URENHA,2021353494,2718600,Serra Azul,SP,-21.266666,-47.583332,16779070882,2040-10-17
320,34.480000,MHz,78,ASSOCIACAO DOS TAXISTAS E RADIO TAXIS MUNIC SA...,2033423808,6560571,São Paulo,SP,-23.474445,-46.656387,67644641000150,2018-03-11
...,...,...,...,...,...,...,...,...,...,...,...,...
380493,27910.000000,kHz,19,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695831232,Seropédica,RJ,-22.769167,-43.695557,29427465000105,2021-08-25
371221,27910.000000,kHz,19,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695826832,Seropédica,RJ,-22.775555,-43.687778,29427465000105,2021-08-25
372214,27910.000000,kHz,19,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695831321,Seropédica,RJ,-22.787777,-43.683613,29427465000105,2021-08-25
352193,27910.000000,kHz,19,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695831305,Seropédica,RJ,-22.789444,-43.664165,29427465000105,2021-08-25


In [ ]:
radcom = read_radcom(path)

In [ ]:
radcom.head()

,Frequência,Fase,Situação,Unidade,Entidade,Fistel,Número da Estação,Município,UF,Latitude,Longitude,CNPJ
0,87.900002,3,K,MHz,ASSOCIACAO RADIO COMUNITARIA EDUCATIVA VERDE A...,50012586269,682737461,Ariquemes,RO,-9.903889,-63.031666,02630029000182
1,104.900002,3,M,MHz,COMEV - RADIO COMUNITARIA CULTURA FM,50011442131,631426833,Cabixi,RO,-13.497778,-60.554165,02355992000103
2,104.900002,3,M,MHz,ASSOCIACAO DA RADIO COMUNITARIA EDUCATIVA CRIS...,50012414999,682610577,Corumbiara,RO,-12.998889,-60.945000,02952610000110
3,87.900002,3,N,MHz,ASSOCIACAO COMUNITARIA ROMIPORA,50011442301,631426850,Espigão D'Oeste,RO,-11.527778,-61.015835,01208823000170
4,105.900002,3,B,MHz,ASSOCIACAO DE RADIO E DIFUSAO COMUNIT. INTERAT...,50011551348,637127374,Jaru,RO,-10.432500,-62.472778,02795569000116


In [ ]:
radcom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4645 entries, 0 to 4644
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Frequência         4645 non-null   float32 
 1   Fase               4645 non-null   category
 2   Situação           4643 non-null   category
 3   Unidade            4645 non-null   category
 4   Entidade           4645 non-null   object  
 5   Fistel             4645 non-null   object  
 6   Número da Estação  4645 non-null   int32   
 7   Município          4645 non-null   object  
 8   UF                 4645 non-null   category
 9   Latitude           4645 non-null   float32 
 10  Longitude          4645 non-null   float32 
 11  CNPJ               4645 non-null   object  
dtypes: category(4), float32(3), int32(1), object(4)
memory usage: 237.9+ KB


In [ ]:
mosaico = read_mosaico(path)

In [ ]:
mosaico.head()

,Serviço,Num_Serviço,Status,Entidade,Fistel,Município,UF,Id,Número_da_Estação,Latitude,Longitude,CNPJ,Validade_RF,Num_Ato,Data_Ato,Frequência,Classe,Canal,Coordenadas_do_Município
0,TV,248,TV-C1,X-MEDIAGROUP S.A.,50410887137,Mâncio Lima,AC,57dbaad053c60,-1,-7.614167,-72.895833,3211814000163,NaN,NaN,NaN,539.0,C,25.0,False
1,TV,248,TV-C4,TELEVISAO OESTE BAIANO LTDA,6030116240,Barreiras,BA,57dbaad0dc4e3,322647029,-12.101389,-44.993611,16395923000120,2023-12-31,6245.0,NaN,79.0,A,5.0,False
2,TV,248,TV-C2,TELEVISAO SANTA CRUZ LTDA,6020355110,Itabuna,BA,57dbaad0eb54a,322623553,-14.779444,-39.262222,13476833000175,2023-12-31,3592.0,2021-05-28,69.0,A,4.0,False
3,TV,248,TV-C3,TV CABRALIA LTDA,6020354903,Itabuna,BA,57dbaad0ef8af,322623537,-14.781670,-39.261670,13494265000135,2023-12-31,7613.0,2018-10-24,177.0,B,7.0,False
4,TV,248,TV-C7,FUNDACAO FUNDESUL,50011828080,Porto Seguro,BA,57dbaad1077a6,637062230,-16.353056,-39.386111,4188244000109,2017-08-20,33606.0,2003-02-10,515.0,C,21.0,False


In [ ]:
mosaico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23503 entries, 0 to 23502
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Serviço                   23503 non-null  object 
 1   Num_Serviço               23503 non-null  int64  
 2   Status                    23503 non-null  object 
 3   Entidade                  23492 non-null  object 
 4   Fistel                    23503 non-null  int64  
 5   Município                 23503 non-null  object 
 6   UF                        23503 non-null  object 
 7   Id                        23503 non-null  object 
 8   Número_da_Estação         23503 non-null  int64  
 9   Latitude                  23503 non-null  float64
 10  Longitude                 23503 non-null  float64
 11  CNPJ                      23503 non-null  int64  
 12  Validade_RF               21223 non-null  object 
 13  Num_Ato                   14073 non-null  float64
 14  Data_A

In [ ]:
mosaicoop = df_optimize(mosaico) #, exclude=['Frequência', 'Latitude', 'Longitude'])

In [ ]:
mosaicoop.head()

,Serviço,Num_Serviço,Status,Entidade,Fistel,Município,UF,Id,Número_da_Estação,Latitude,Longitude,CNPJ,Validade_RF,Num_Ato,Data_Ato,Frequência,Classe,Canal,Coordenadas_do_Município
0,TV,248,TV-C1,X-MEDIAGROUP S.A.,50410887137,Mâncio Lima,AC,57dbaad053c60,-1,-7.614167,-72.895836,3211814000163,NaN,NaN,NaN,539.0,C,25.0,False
1,TV,248,TV-C4,TELEVISAO OESTE BAIANO LTDA,6030116240,Barreiras,BA,57dbaad0dc4e3,322647029,-12.101389,-44.993610,16395923000120,2023-12-31,6245.0,NaN,79.0,A,5.0,False
2,TV,248,TV-C2,TELEVISAO SANTA CRUZ LTDA,6020355110,Itabuna,BA,57dbaad0eb54a,322623553,-14.779445,-39.262222,13476833000175,2023-12-31,3592.0,2021-05-28,69.0,A,4.0,False
3,TV,248,TV-C3,TV CABRALIA LTDA,6020354903,Itabuna,BA,57dbaad0ef8af,322623537,-14.781670,-39.261669,13494265000135,2023-12-31,7613.0,2018-10-24,177.0,B,7.0,False
4,TV,248,TV-C7,FUNDACAO FUNDESUL,50011828080,Porto Seguro,BA,57dbaad1077a6,637062230,-16.353056,-39.386112,4188244000109,2017-08-20,33606.0,2003-02-10,515.0,C,21.0,False


In [ ]:
mosaicoop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23503 entries, 0 to 23502
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Serviço                   23503 non-null  category
 1   Num_Serviço               23503 non-null  int16   
 2   Status                    23503 non-null  category
 3   Entidade                  23492 non-null  category
 4   Fistel                    23503 non-null  int64   
 5   Município                 23503 non-null  category
 6   UF                        23503 non-null  category
 7   Id                        23503 non-null  object  
 8   Número_da_Estação         23503 non-null  int32   
 9   Latitude                  23503 non-null  float32 
 10  Longitude                 23503 non-null  float32 
 11  CNPJ                      23503 non-null  int64   
 12  Validade_RF               21223 non-null  category
 13  Num_Ato                   14073 non-null  floa

In [ ]:
df = update_base(path)

In [ ]:
df = df_optimize(df, exclude=['Frequência', 'Latitude', 'Longitude', 'Validade_RF', 'Data_Ato'])

In [ ]:
df.head()

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato
0,0.54,205,AM-C3,B,RADIO SEPE TIARAJU LTDA,3008013783,9546529,Santo Ângelo,RS,-28.283333,-54.266667,2026-09-06,10865.0,2017-08-30
1,0.54,205,AM-C3,B,RADIO RIVIERA LTDA,13008004962,323010458,Goiânia,GO,-16.715556,-49.245278,2023-11-01,1008.0,2018-03-08
2,0.54,205,AM-C1,B,MCC-PARTICIPACOES LTDA,50411625144,-1,Uberaba,MG,-19.750000,-47.933333,NaN,NaN,NaN
3,0.54,205,AM-C7,C,RADIO PRIMEIRO DE JULHO LTDA,10020061986,322880360,Água Branca,PI,-5.896389,-42.636667,NaN,NaN,NaN
4,0.54,205,AM-C7,C,RADIO NOVA SUMARE LTDA,2008029093,7805810,Sumaré,SP,-22.833333,-47.283333,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432572 entries, 0 to 432571
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Frequência         432572 non-null  float64
 1   Num_Serviço        432572 non-null  int64  
 2   Status             28148 non-null   object 
 3   Classe             28119 non-null   object 
 4   Entidade           432561 non-null  object 
 5   Fistel             432572 non-null  object 
 6   Número_da_Estação  432572 non-null  int64  
 7   Município          432572 non-null  object 
 8   UF                 432572 non-null  object 
 9   Latitude           432572 non-null  float64
 10  Longitude          432572 non-null  float64
 11  Validade_RF        430292 non-null  object 
 12  Num_Ato            423142 non-null  object 
 13  Data_Ato           422139 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 46.2+ MB


In [ ]:
df.to_feather(f'{path}/base.fth')

ArrowInvalid: ("Could not convert '50402159403' with type str: tried to convert to int", 'Conversion failed for column Fistel with type object')

In [ ]:
#df = update_mosaico(r'D:\OneDrive - ANATEL\BaseDados')

In [ ]:
# df.info()

# df.head()

# radcom = read_radcom(r'D:\OneDrive - ANATEL\BaseDados')

# radcom.info()

# stel = read_stel(r'D:\OneDrive - ANATEL\BaseDados')

# stel.info()

In [ ]:
#e = read_estações(r'D:\OneDrive - ANATEL\BaseDados\estações.zip')

In [ ]:
#pb = read_plano_basico(r'D:\OneDrive - ANATEL\BaseDados\Canais.zip')

In [ ]:
#pb.to_feather(r'D:\OneDrive - ANATEL\BaseDados\plano_basico.fth')

In [ ]:
#df = e.merge(pb, on="Id", how="left")

In [ ]:
#df = clean_merge(r'D:\OneDrive - ANATEL\BaseDados', df)

In [ ]:
# df['Canal'] = df['Canal'].astype()
# df.to_feather(r'D:\OneDrive - ANATEL\BaseDados\mosaico.fth')

In [ ]:
from nbdev.export import notebook2script; notebook2script()

In [ ]:
# import requests

# r = requests.get('https://anatel365-my.sharepoint.com/:x:/g/personal/rsilva_anatel_gov_br/ESjSacOqe3xEpodgo9-cRW0ByQD_7fIn_NVpP1A6gz7V0Q?download=1')

# r

# p = Path(r'D:\OneDrive - ANATEL\BaseDados\AnatelDB.xlsx')

# p.write_bytes(r.content)

# d = requests.get('https://anatel365-my.sharepoint.com/:x:/g/personal/rsilva_anatel_gov_br/ESjSacOqe3xEpodgo9-cRW0BCaLnzQUHFTH8vdkH7sZwkg?download=1')

# p = Path(r'D:\OneDrive - ANATEL\BaseDados\Teste.xlsx')

# p.write_bytes(d.content)

In [ ]:
#stel = read_stel(r'D:\OneDrive - ANATEL\BaseDados')

In [ ]:
# stel

# stel.Serviço.value_counts()

# stel.loc[stel.Serviço == '507', 'Entidade'].value_counts()